# Generados de PUZZLE

In [41]:
import random

def generar_puzzle():
    """Genera un puzzle deslizante 3x3 aleatorio como tupla de tuplas."""
    nums = list(range(9))  # Números del 0 al 8
    random.shuffle(nums)   # Mezcla aleatoriamente
    est = tuple(tuple(nums[i * 3:(i + 1) * 3]) for i in range(3))
    return est

# Ejemplo de uso:
estado_inicial = generar_puzzle()

# estado_inicial= ( #puzzle de prueba que se puede resolver y no tardará mucho
#     (1, 2, 3),
#     (7, 0, 4),
#     (8, 6, 5)
# )

estado_objetivo = (
    (1, 2, 3),
    (4, 5, 6),
    (7, 8, 0)
)

print("Estado inicial:")
for fila in estado_inicial:
    print(' '.join(str(x) if x != 0 else ' ' for x in fila))

Estado inicial:
5 1 6
8 4 3
2   7


# Solucion de puzzle usando Cola (BFS/FIFO)

In [42]:
class Nodo():
    def __init__(self, estado, padre, accion):
        self.estado = estado  # Estado es una tupla de tuplas (matriz 3x3)
        self.padre = padre
        self.accion = accion

class Frontera():
    def __init__(self):
        self.frontera =[] # Inicializa la frontera como una lista vacía

    def empty(self):
        return (len(self.frontera) == 0) # Verifica si la frontera está vacía

    def add(self, nodo):
        self.frontera.append(nodo) # Agrega un nodo a la frontera

    def eliminar(self):
        # LIFO o FIFO 
        pass 

    def contiene_estado(self, estado):
        return any(nodo.estado == estado for nodo in self.frontera) # Verifica si un estado ya está en la frontera

class Pila(Frontera):
    def eliminar(self):
        # Termina la busqueda si la frontera esta vacia
        if self.empty():
            raise Exception("Frontera vacia")
        else:
            # Guardamos el ultimo item en la lista
            # (el cual es el nodo recientemente añadido)
            nodo = self.frontera[-1]
            # Guardamos todos los items excepto el
            # ultimo (eliminamos)
            self.frontera = self.frontera[:-1]
            return nodo

class Cola(Frontera):
    def eliminar(self):
        # Termina la busqueda si la frontera esta vacia
        if self.empty():
            raise Exception("Frontera vacia")
        else:
            # Guardamos el primer item en la lista
            # (el cual es el nodo añadido de primero)
            nodo = self.frontera[0]
            # Guardamos todos los items excepto el
            # primero (eliminamos)
            self.frontera = self.frontera[1:]
            return nodo

# Clase para el puzzle deslizante 3x3
class PuzzleDeslizante():
    def __init__(self, estado_inicial, estado_objetivo):
        self.inicio = estado_inicial
        self.objetivo = estado_objetivo
        self.solucion = None

    def print(self, estado=None): # Imprime el estado del puzzle
        if estado is None: # Si no se proporciona un estado, usa el inicial
            estado = self.inicio
        for fila in estado:
            print(' '.join(str(x) if x != 0 else ' ' for x in fila)) # Imprime cada fila
        print()

    def encontrar_vacio(self, estado): # Encuentra la posición del espacio vacío (0)
        for i in range(3):
            for j in range(3):
                if estado[i][j] == 0:
                    return (i, j)

    def mover(self, estado, direccion):
        i, j = self.encontrar_vacio(estado)
        nuevo_estado = [list(fila) for fila in estado] # Copia el estado actual a una lista mutable

        if direccion == "up" and i > 0:
            nuevo_estado[i][j], nuevo_estado[i-1][j] = nuevo_estado[i-1][j], nuevo_estado[i][j]

        elif direccion == "down" and i < 2:
            nuevo_estado[i][j], nuevo_estado[i+1][j] = nuevo_estado[i+1][j], nuevo_estado[i][j]

        elif direccion == "left" and j > 0:
            nuevo_estado[i][j], nuevo_estado[i][j-1] = nuevo_estado[i][j-1], nuevo_estado[i][j]

        elif direccion == "right" and j < 2:
            nuevo_estado[i][j], nuevo_estado[i][j+1] = nuevo_estado[i][j+1], nuevo_estado[i][j]

        else:
            return None
        return tuple(tuple(fila) for fila in nuevo_estado) # Convierte la lista de nuevo a tupla para mantener la inmutabilidad

    def vecinos(self, estado): # Genera los estados vecinos posibles
        acciones = ["up", "down", "left", "right"]
        resultado = []
        for accion in acciones:
            nuevo_estado = self.mover(estado, accion)
            if nuevo_estado is not None:
                resultado.append((accion, nuevo_estado))
        return resultado

    def soluble(self, estado): # Verifica si el puzzle es soluble
        plano = sum(estado, ())  # Convierte la matriz en una tupla plana
        inversiones = 0
        lista = [x for x in plano if x != 0]
        for i in range(len(lista)):
            for j in range(i + 1, len(lista)): # J empieza en i+1 para evitar contar la misma inversión dos veces
                if lista[i] > lista[j]:
                    inversiones += 1
        return inversiones % 2 == 0 # El número de inversiones debe ser par para que el puzzle sea resoluble

    def solve(self):
        if not self.soluble(self.inicio): # Verifica si el puzzle es resoluble usando la función 'soluble'
            print("Este puzzle no es resoluble. :(")
            return
        else:
            print("Este puzzle es resoluble. :)")
            print("Buscando solución...")

        self.num_explorados = 0
        start = Nodo(estado=self.inicio, padre=None, accion=None) # Nodo inicial
        frontera = Cola()  # Usa Cola para BFS (solución más corta) o Pila para DFS (exploración profunda)
        frontera.add(start) # Agrega el nodo inicial a la frontera
        self.explorado = set()

        while True:
            if frontera.empty(): # Verifica si la frontera está vacía
                raise Exception("No hay solución")
            nodo = frontera.eliminar() # Extrae el nodo de la frontera dependiendo de la estrategia (FIFO o LIFO)
            self.num_explorados += 1

            if nodo.estado == self.objetivo: 
                acciones = []
                estados = []
                while nodo.padre is not None: # Recorre el árbol de nodos desde el nodo objetivo hasta el inicial
                    acciones.append(nodo.accion)
                    estados.append(nodo.estado)
                    nodo = nodo.padre
                acciones.reverse()
                estados.reverse()
                self.solucion = (acciones, estados)
                return # Devuelve la solución encontrada

            self.explorado.add(nodo.estado) # Agrega el nodo actual a la lista de explorados
            for accion, estado in self.vecinos(nodo.estado): # Genera los vecinos del nodo actual
                if not frontera.contiene_estado(estado) and estado not in self.explorado: # Verifica si el estado ya fue explorado o está en la frontera
                    hijo = Nodo(estado=estado, padre=nodo, accion=accion)
                    frontera.add(hijo) # Agrega el nuevo nodo a la frontera

puzzle = PuzzleDeslizante(estado_inicial, estado_objetivo)
print("Puzzle inicial:")
puzzle.print()
puzzle.solve()

if puzzle.solucion:
    print("Estados explorados:", puzzle.num_explorados)
    print("Solución (acciones):", puzzle.solucion[0])
    print("cantidad de pasos:", len(puzzle.solucion[0]))
    print("Estados intermedios:")
    for estado in puzzle.solucion[1]:
        puzzle.print(estado)

Puzzle inicial:
5 1 6
8 4 3
2   7

Este puzzle es resoluble. :)
Buscando solución...
Estados explorados: 107727
Solución (acciones): ['up', 'left', 'up', 'right', 'right', 'down', 'left', 'up', 'left', 'down', 'down', 'right', 'right', 'up', 'left', 'left', 'up', 'right', 'down', 'left', 'down', 'right', 'right']
cantidad de pasos: 23
Estados intermedios:
5 1 6
8   3
2 4 7

5 1 6
  8 3
2 4 7

  1 6
5 8 3
2 4 7

1   6
5 8 3
2 4 7

1 6  
5 8 3
2 4 7

1 6 3
5 8  
2 4 7

1 6 3
5   8
2 4 7

1   3
5 6 8
2 4 7

  1 3
5 6 8
2 4 7

5 1 3
  6 8
2 4 7

5 1 3
2 6 8
  4 7

5 1 3
2 6 8
4   7

5 1 3
2 6 8
4 7  

5 1 3
2 6  
4 7 8

5 1 3
2   6
4 7 8

5 1 3
  2 6
4 7 8

  1 3
5 2 6
4 7 8

1   3
5 2 6
4 7 8

1 2 3
5   6
4 7 8

1 2 3
  5 6
4 7 8

1 2 3
4 5 6
  7 8

1 2 3
4 5 6
7   8

1 2 3
4 5 6
7 8  



# Solucion de puzzle usando Pila (DFS/LIFO)

In [43]:
class Nodo():
    def __init__(self, estado, padre, accion):
        self.estado = estado  # Estado es una tupla de tuplas (matriz 3x3)
        self.padre = padre
        self.accion = accion

class Frontera():
    def __init__(self):
        self.frontera =[] # Inicializa la frontera como una lista vacía

    def empty(self):
        return (len(self.frontera) == 0) # Verifica si la frontera está vacía

    def add(self, nodo):
        self.frontera.append(nodo) # Agrega un nodo a la frontera

    def eliminar(self):
        # LIFO o FIFO 
        pass 

    def contiene_estado(self, estado):
        return any(nodo.estado == estado for nodo in self.frontera) # Verifica si un estado ya está en la frontera

class Pila(Frontera):
    def eliminar(self):
        # Termina la busqueda si la frontera esta vacia
        if self.empty():
            raise Exception("Frontera vacia")
        else:
            # Guardamos el ultimo item en la lista
            # (el cual es el nodo recientemente añadido)
            nodo = self.frontera[-1]
            # Guardamos todos los items excepto el
            # ultimo (eliminamos)
            self.frontera = self.frontera[:-1]
            return nodo

class Cola(Frontera):
    def eliminar(self):
        # Termina la busqueda si la frontera esta vacia
        if self.empty():
            raise Exception("Frontera vacia")
        else:
            # Guardamos el primer item en la lista
            # (el cual es el nodo añadido de primero)
            nodo = self.frontera[0]
            # Guardamos todos los items excepto el
            # primero (eliminamos)
            self.frontera = self.frontera[1:]
            return nodo

# Clase para el puzzle deslizante 3x3
class PuzzleDeslizante():
    def __init__(self, estado_inicial, estado_objetivo):
        self.inicio = estado_inicial
        self.objetivo = estado_objetivo
        self.solucion = None

    def print(self, estado=None): # Imprime el estado del puzzle
        if estado is None: # Si no se proporciona un estado, usa el inicial
            estado = self.inicio
        for fila in estado:
            print(' '.join(str(x) if x != 0 else ' ' for x in fila)) # Imprime cada fila
        print()

    def encontrar_vacio(self, estado): # Encuentra la posición del espacio vacío (0)
        for i in range(3):
            for j in range(3):
                if estado[i][j] == 0:
                    return (i, j)

    def mover(self, estado, direccion):
        i, j = self.encontrar_vacio(estado)
        nuevo_estado = [list(fila) for fila in estado] # Copia el estado actual a una lista mutable

        if direccion == "up" and i > 0:
            nuevo_estado[i][j], nuevo_estado[i-1][j] = nuevo_estado[i-1][j], nuevo_estado[i][j]

        elif direccion == "down" and i < 2:
            nuevo_estado[i][j], nuevo_estado[i+1][j] = nuevo_estado[i+1][j], nuevo_estado[i][j]

        elif direccion == "left" and j > 0:
            nuevo_estado[i][j], nuevo_estado[i][j-1] = nuevo_estado[i][j-1], nuevo_estado[i][j]

        elif direccion == "right" and j < 2:
            nuevo_estado[i][j], nuevo_estado[i][j+1] = nuevo_estado[i][j+1], nuevo_estado[i][j]

        else:
            return None
        return tuple(tuple(fila) for fila in nuevo_estado) # Convierte la lista de nuevo a tupla para mantener la inmutabilidad

    def vecinos(self, estado): # Genera los estados vecinos posibles
        acciones = ["up", "down", "left", "right"]
        resultado = []
        for accion in acciones:
            nuevo_estado = self.mover(estado, accion)
            if nuevo_estado is not None:
                resultado.append((accion, nuevo_estado))
        return resultado

    def soluble(self, estado): # Verifica si el puzzle es soluble
        plano = sum(estado, ())  # Convierte la matriz en una tupla plana
        inversiones = 0
        lista = [x for x in plano if x != 0]
        for i in range(len(lista)):
            for j in range(i + 1, len(lista)): # J empieza en i+1 para evitar contar la misma inversión dos veces
                if lista[i] > lista[j]:
                    inversiones += 1
        return inversiones % 2 == 0 # El número de inversiones debe ser par para que el puzzle sea resoluble

    def solve(self):
        if not self.soluble(self.inicio): # Verifica si el puzzle es resoluble usando la función 'soluble'
            print("Este puzzle no es resoluble. :(")
            return
        else:
            print("Este puzzle es resoluble. :)")
            print("Buscando solución...")

        self.num_explorados = 0
        start = Nodo(estado=self.inicio, padre=None, accion=None) # Nodo inicial
        frontera = Pila()  # Usa Cola para BFS (solución más corta) o Pila para DFS (exploración profunda)
        frontera.add(start) # Agrega el nodo inicial a la frontera
        self.explorado = set()

        while True:
            if frontera.empty():
                raise Exception("No hay solución")
            nodo = frontera.eliminar()
            self.num_explorados += 1

            if nodo.estado == self.objetivo:
                acciones = []
                estados = []
                while nodo.padre is not None:
                    acciones.append(nodo.accion)
                    estados.append(nodo.estado)
                    nodo = nodo.padre
                acciones.reverse()
                estados.reverse()
                self.solucion = (acciones, estados)
                return

            self.explorado.add(nodo.estado)
            for accion, estado in self.vecinos(nodo.estado):
                if not frontera.contiene_estado(estado) and estado not in self.explorado:
                    hijo = Nodo(estado=estado, padre=nodo, accion=accion)
                    frontera.add(hijo)

puzzle = PuzzleDeslizante(estado_inicial, estado_objetivo)
print("Puzzle inicial:")
puzzle.print()
puzzle.solve()

if puzzle.solucion:
    print("Estados explorados:", puzzle.num_explorados)
    print("Solución (acciones):", puzzle.solucion[0])
    print("cantidad de pasos:", len(puzzle.solucion[0]))
    print("Estados intermedios:")
    for estado in puzzle.solucion[1]:
        puzzle.print(estado)

Puzzle inicial:
5 1 6
8 4 3
2   7

Este puzzle es resoluble. :)
Buscando solución...
Estados explorados: 10601
Solución (acciones): ['right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'up', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'up', 'right', 'down

# Solucion de puzzle usando A*

In [44]:
class Nodo():
    def __init__(self, estado, padre, accion, costo=0, heuristica=0):
        self.estado = estado  # Estado es una tupla de tuplas (matriz 3x3)
        self.padre = padre
        self.accion = accion
        self.costo = costo  # Costo del camino hasta este nodo
        self.heuristica = heuristica  # Heurística para el nodo (si se usa A* o similar)
        self.f = costo + heuristica  # Valor total (costo + heurística)

class Frontera():
    def __init__(self):
        self.frontera =[] # Inicializa la frontera como una lista vacía

    def empty(self):
        return (len(self.frontera) == 0) # Verifica si la frontera está vacía

    def add(self, nodo):
        self.frontera.append(nodo) # Agrega un nodo a la frontera

    def eliminar(self):
        # LIFO, FIFO o A* (según la implementación)
        pass 

    def contiene_estado(self, estado):
        return any(nodo.estado == estado for nodo in self.frontera) # Verifica si un estado ya está en la frontera

class ColaPrioridad(Frontera):
    def eliminar(self):
        # Termina la busqueda si la frontera esta vacia
        if self.empty():
            raise Exception("Frontera vacia")
        else:
            # Encontrar el nodo con menor f = g + h
            mejor_indice = 0
            for i in range(len(self.frontera)):
                if self.frontera[i].f < self.frontera[mejor_indice].f:
                    mejor_indice = i
            nodo = self.frontera[mejor_indice]
            # Eliminamos el nodo de la frontera
            self.frontera.pop(mejor_indice)
            return nodo

class Pila(Frontera):
    def eliminar(self):
        # Termina la busqueda si la frontera esta vacia
        if self.empty():
            raise Exception("Frontera vacia")
        else:
            # Guardamos el ultimo item en la lista
            # (el cual es el nodo recientemente añadido)
            nodo = self.frontera[-1]
            # Guardamos todos los items excepto el
            # ultimo (eliminamos)
            self.frontera = self.frontera[:-1]
            return nodo

class Cola(Frontera):
    def eliminar(self):
        # Termina la busqueda si la frontera esta vacia
        if self.empty():
            raise Exception("Frontera vacia")
        else:
            # Guardamos el primer item en la lista
            # (el cual es el nodo añadido de primero)
            nodo = self.frontera[0]
            # Guardamos todos los items excepto el
            # primero (eliminamos)
            self.frontera = self.frontera[1:]
            return nodo

# Clase para el puzzle deslizante 3x3
class PuzzleDeslizante():
    def __init__(self, estado_inicial, estado_objetivo):
        self.inicio = estado_inicial
        self.objetivo = estado_objetivo
        self.solucion = None

    def print(self, estado=None): # Imprime el estado del puzzle
        if estado is None: # Si no se proporciona un estado, usa el inicial
            estado = self.inicio
        for fila in estado:
            print(' '.join(str(x) if x != 0 else ' ' for x in fila)) # Imprime cada fila
        print()

    def encontrar_vacio(self, estado): # Encuentra la posición del espacio vacío (0)
        for i in range(3):
            for j in range(3):
                if estado[i][j] == 0:
                    return (i, j)

    def mover(self, estado, direccion):
        i, j = self.encontrar_vacio(estado)
        nuevo_estado = [list(fila) for fila in estado] # Copia el estado actual a una lista mutable

        if direccion == "up" and i > 0:
            nuevo_estado[i][j], nuevo_estado[i-1][j] = nuevo_estado[i-1][j], nuevo_estado[i][j]

        elif direccion == "down" and i < 2:
            nuevo_estado[i][j], nuevo_estado[i+1][j] = nuevo_estado[i+1][j], nuevo_estado[i][j]

        elif direccion == "left" and j > 0:
            nuevo_estado[i][j], nuevo_estado[i][j-1] = nuevo_estado[i][j-1], nuevo_estado[i][j]

        elif direccion == "right" and j < 2:
            nuevo_estado[i][j], nuevo_estado[i][j+1] = nuevo_estado[i][j+1], nuevo_estado[i][j]

        else:
            return None
        return tuple(tuple(fila) for fila in nuevo_estado) # Convierte la lista de nuevo a tupla para mantener la inmutabilidad

    def vecinos(self, estado): # Genera los estados vecinos posibles
        acciones = ["up", "down", "left", "right"]
        resultado = []
        for accion in acciones:
            nuevo_estado = self.mover(estado, accion)
            if nuevo_estado is not None:
                resultado.append((accion, nuevo_estado))
        return resultado

    def soluble(self, estado): # Verifica si el puzzle es soluble
        plano = sum(estado, ())  # Convierte la matriz en una tupla plana
        inversiones = 0
        lista = [x for x in plano if x != 0]
        for i in range(len(lista)):
            for j in range(i + 1, len(lista)): # J empieza en i+1 para evitar contar la misma inversión dos veces
                if lista[i] > lista[j]:
                    inversiones += 1
        return inversiones % 2 == 0 # El número de inversiones debe ser par para que el puzzle sea resoluble

    # Función que calcula la distancia de Manhattan
    def manhattan(self, estado):
        """Calcula la distancia de Manhattan para cada ficha."""
        distancia = 0
        # Para cada número del 1-8, calculamos su posición correcta
        for i in range(3):
            for j in range(3):
                valor = estado[i][j]
                if valor != 0:  # Ignoramos el espacio vacío
                    # Calcular posición correcta del valor (en base 0)
                    fila_objetivo = (valor - 1) // 3
                    columna_objetivo = (valor - 1) % 3
                    # Sumar la distancia Manhattan absoluta
                    distancia += abs(i - fila_objetivo) + abs(j - columna_objetivo)
        return distancia

    def solve(self):
        if not self.soluble(self.inicio): # Verifica si el puzzle es resoluble usando la función 'soluble'
            print("Este puzzle no es resoluble. :(")
            return
        else:
            print("Este puzzle es resoluble. :)")
            print("Buscando solución...")

        self.num_explorados = 0
        # Inicializar con heurística calculada
        h_inicio = self.manhattan(self.inicio)
        start = Nodo(estado=self.inicio, padre=None, accion=None, costo=0, heuristica=h_inicio)
        frontera = ColaPrioridad()  # Usa Cola para BFS (solución más corta), Pila para DFS (exploración profunda) o ColaPrioridad para A*
        frontera.add(start) # Agrega el nodo inicial a la frontera
        self.explorado = set()

        while True:
            if frontera.empty():
                raise Exception("No hay solución")
            nodo = frontera.eliminar()
            self.num_explorados += 1

            if nodo.estado == self.objetivo:
                acciones = []
                estados = []
                while nodo.padre is not None:
                    acciones.append(nodo.accion)
                    estados.append(nodo.estado)
                    nodo = nodo.padre
                acciones.reverse()
                estados.reverse()
                self.solucion = (acciones, estados)
                return

            self.explorado.add(nodo.estado)
            for accion, estado in self.vecinos(nodo.estado):
                if not frontera.contiene_estado(estado) and estado not in self.explorado:
                    # Calcular g(n) = costo hasta ahora + 1
                    costo_g = nodo.costo + 1
                    # Calcular h(n) = distancia Manhattan
                    heuristica_h = self.manhattan(estado)
                    # Crear nuevo nodo con f = g + h
                    hijo = Nodo(estado=estado, padre=nodo, accion=accion,
                                costo=costo_g, heuristica=heuristica_h)
                    frontera.add(hijo)

puzzle = PuzzleDeslizante(estado_inicial, estado_objetivo)
print("Puzzle inicial:")
puzzle.print()
puzzle.solve()

if puzzle.solucion:
    print("Estados explorados:", puzzle.num_explorados)
    print("Solución (acciones):", puzzle.solucion[0])
    print("cantidad de pasos:", len(puzzle.solucion[0]))
    print("Estados intermedios:")
    for estado in puzzle.solucion[1]:
        puzzle.print(estado)

Puzzle inicial:
5 1 6
8 4 3
2   7

Este puzzle es resoluble. :)
Buscando solución...
Estados explorados: 1912
Solución (acciones): ['left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'up', 'right', 'down', 'right', 'down', 'left', 'up', 'right', 'up', 'left', 'down', 'left', 'down', 'right', 'right']
cantidad de pasos: 23
Estados intermedios:
5 1 6
8 4 3
  2 7

5 1 6
  4 3
8 2 7

5 1 6
4   3
8 2 7

5 1 6
4 3  
8 2 7

5 1 6
4 3 7
8 2  

5 1 6
4 3 7
8   2

5 1 6
4 3 7
  8 2

5 1 6
  3 7
4 8 2

  1 6
5 3 7
4 8 2

1   6
5 3 7
4 8 2

1 3 6
5   7
4 8 2

1 3 6
5 7  
4 8 2

1 3 6
5 7 2
4 8  

1 3 6
5 7 2
4   8

1 3 6
5   2
4 7 8

1 3 6
5 2  
4 7 8

1 3  
5 2 6
4 7 8

1   3
5 2 6
4 7 8

1 2 3
5   6
4 7 8

1 2 3
  5 6
4 7 8

1 2 3
4 5 6
  7 8

1 2 3
4 5 6
7   8

1 2 3
4 5 6
7 8  

